In [2]:
import yfinance as yf
import pandas as pd
import numpy as np

In [3]:
def calculate_max_drawdown(df: pd.DataFrame):
    df_copy = df.copy()
    df_copy = df_copy.sort_index()

    df_copy['Cumulative Max'] = df_copy['Close'].cummax()
    df_copy['Drawdown'] = df_copy['Close'] / df_copy['Cumulative Max'] - 1
    max_drawdown = df_copy['Drawdown'].min()

    max_drawdown_date = df_copy['Drawdown'].idxmin()
    cumulative_max_date = df_copy.loc[:max_drawdown_date, 'Cumulative Max'].idxmax()

    print(f"The maximum drawdown is {max_drawdown:.2%}")
    print(f"The cumulative maximum was reached on {cumulative_max_date.date()}")
    print(f"The maximum drawdown occurred on {max_drawdown_date.date()}")

## IXC Max Drawdown

In [4]:
ixc = yf.Ticker("IXC")
ixc_data = ixc.history(period="2y")

In [5]:
calculate_max_drawdown(ixc_data)

The maximum drawdown is -17.57%
The cumulative maximum was reached on 2022-08-29
The maximum drawdown occurred on 2022-09-26


## TQQ Max Drawdown

In [6]:
tqqq = yf.Ticker("TQQQ")
tqqq_data = tqqq.history(period="2y")

In [7]:
calculate_max_drawdown(tqqq_data)

The maximum drawdown is -58.23%
The cumulative maximum was reached on 2022-08-15
The maximum drawdown occurred on 2022-12-28


## Portfolio Max Drawdown

In [8]:
def calculate_max_drawdown_for_portfolio(tqqq_df: pd.DataFrame, ixc_df: pd.DataFrame):
    tqqq_df = tqqq_df.sort_index()
    ixc_df = ixc_df.sort_index()

    combined_df = pd.DataFrame(index=tqqq_df.index)
    combined_df['TQQQ_Close'] = tqqq_df['Close']
    combined_df['IXC_Close'] = ixc_df['Close']

    combined_df = combined_df.dropna()

    # Calculate the portfolio value with a 50:50 split
    combined_df['Portfolio_Value'] = 0.5 * combined_df['TQQQ_Close'] + 0.5 * combined_df['IXC_Close']
    combined_df['Cumulative Max'] = combined_df['Portfolio_Value'].cummax()

    # Calculate the drawdown
    combined_df['Drawdown'] = combined_df['Portfolio_Value'] / combined_df['Cumulative Max'] - 1

    # Calculate the maximum drawdown
    max_drawdown = combined_df['Drawdown'].min()

    # Find the date of the maximum drawdown
    max_drawdown_date = combined_df['Drawdown'].idxmin()

    # Find the date of the cumulative maximum before the maximum drawdown
    cumulative_max_date = combined_df.loc[:max_drawdown_date, 'Cumulative Max'].idxmax()

    print(f"The maximum drawdown for the portfolio is {max_drawdown:.2%}")
    print(f"The cumulative maximum was reached on {cumulative_max_date.date()}")
    print(f"The maximum drawdown occurred on {max_drawdown_date.date()}")

In [9]:
calculate_max_drawdown_for_portfolio(tqqq_df=tqqq_data, ixc_df=ixc_data)

The maximum drawdown for the portfolio is -30.21%
The cumulative maximum was reached on 2022-08-15
The maximum drawdown occurred on 2022-09-30


In [18]:
def calculate_max_drawdown(portfolio_values):
    cumulative_max = portfolio_values.cummax()
    drawdown = portfolio_values / cumulative_max - 1
    max_drawdown = drawdown.min()
    return max_drawdown

def find_optimal_split(tqqq_df: pd.DataFrame, ixc_df: pd.DataFrame):
    tqqq_df = tqqq_df.sort_index()
    ixc_df = ixc_df.sort_index()

    combined_df = pd.DataFrame(index=tqqq_df.index)
    combined_df['TQQQ_Close'] = tqqq_df['Close']
    combined_df['IXC_Close'] = ixc_df['Close']
    combined_df = combined_df.dropna()

    best_split = None
    lowest_max_drawdown = -float('inf')

    for split in np.arange(0, 1.01, 0.01):
        combined_df['Portfolio_Value'] = split * combined_df['TQQQ_Close'] + (1 - split) * combined_df['IXC_Close']
        max_drawdown = calculate_max_drawdown(combined_df['Portfolio_Value'])

        if max_drawdown > lowest_max_drawdown:
            lowest_max_drawdown = max_drawdown
            best_split = split

    print(f"The optimal split ratio is {best_split:.2f} for TQQQ and {1 - best_split:.2f} for IXC")
    print(f"The lowest maximum drawdown is {lowest_max_drawdown:.2%}")

In [22]:
find_optimal_split(tqqq_df=tqqq_data, ixc_df=ixc_data)

The optimal split ratio is 0.00 for TQQQ and 1.00 for IXC
The lowest maximum drawdown is -17.57%


In [20]:
def calculate_correlation(tqqq_df: pd.DataFrame, ixc_df: pd.DataFrame):
    # Ensure both DataFrames are sorted by date
    tqqq_df = tqqq_df.sort_index()
    ixc_df = ixc_df.sort_index()

    # Align the data by date
    combined_df = pd.DataFrame(index=tqqq_df.index)
    combined_df['TQQQ_Close'] = tqqq_df['Close']
    combined_df['IXC_Close'] = ixc_df['Close']

    # Drop rows with missing data
    combined_df = combined_df.dropna()

    # Calculate the correlation
    correlation = combined_df['TQQQ_Close'].corr(combined_df['IXC_Close'])

    print(f"The correlation between TQQQ and IXC is {correlation:.2f}")

In [21]:
calculate_correlation(tqqq_df=tqqq_data, ixc_df=ixc_data)

The correlation between TQQQ and IXC is 0.66
